In [14]:
import rdflib
from wikidataintegrator import wdi_core, wdi_login
from getpass import getpass
import pandas as pd
import sys
import traceback
from ShExJSG import Schema, ShExC
from ShExJSG.ShExJ import Shape, IRIREF, TripleConstraint, NodeConstraint, EachOf
from pyjsg.jsglib.loader import is_valid
from rdflib import Namespace, Graph, RDFS

wbstack = "lux-saf-beta"
wikibase = "https://{}.wiki.opencura.com/".format(wbstack)
api = "https://{}.wiki.opencura.com/w/api.php".format(wbstack)
sparql = "https://{}.wiki.opencura.com/query/sparql".format(wbstack)
entityUri = wikibase.replace("https:", "http:")+"entity/"
cidocUri = "http://erlangen-crm.org/current/"
WBUSER = "Andrawaag"
WBPASS = getpass()
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)

········
Successfully logged in as Andrawaag


In [15]:
qid = dict()
query = "PREFIX wdt: <http://{}.wiki.opencura.com/prop/direct/> SELECT ?item ?label WHERE {{?item rdfs:label ?label }}".format(wbstack)
wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql)
for index, row in wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql).iterrows():
    qid[row["label"]] =row["item"].replace(entityUri, "")

In [16]:
query = f"""
    PREFIX wde: <{wikibase.replace("https:", "http:")}prop/direct/>
    SELECT * WHERE {{
       ?wikibaseprop wde:{qid["domain"]} ?domain ;
                    wde:{qid["range"]} ?range ;
                    wde:{qid["property"]} ?property .
       ?domain wde:{qid["exact match"]} ?cidocdomain ; rdfs:label ?domainlabel .
       ?range wde:{qid["exact match"]} ?cidocrange ; rdfs:label ?rangelabel .
       ?property wde:{qid["exact match"]} ?cidocproperty ; rdfs:label ?propertylabel .
      }} ORDER BY ?wikibaseprop
"""
results = wdi_core.WDItemEngine.execute_sparql_query(query, endpoint=sparql, as_dataframe=True)
mappings = results.replace(entityUri,'', regex=True)

In [78]:
shexOutput = f"""
        PREFIX wdt: <{wikibase}prop/direct/>
        PREFIX wd: <{wikibase}entity/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX saflux: <{wikibase}entity/>
        PREFIX p: <{wikibase}prop/>
        PREFIX cidoc: <{cidocUri}>
"""

for prop_id in properties:
    prop = localEntityEngine(wd_item_id=prop_id)
    prop_dict = prop.get_wd_json_representation()
    values = []
    for lang in prop_dict["labels"].keys():
        values.append("\""+prop_dict["labels"][lang]["value"]+"\"@"+lang)

    tuple_mappings = mappings[mappings["wikibaseprop"]==prop_id]
    print(len(tuple_mappings))
    if len(tuple_mappings) == 0:
         shexOutput += f"""
            wdt:{prop_id} {{
            rdfs:label [{" ".join(values)}]+;
            }}"""
    else:
        shexOutput += f"""
            wdt:{prop_id} {{
              rdfs:label [{" ".join(values)}]+;
              wdt:{qid["domain"]} {{ rdfs:label [ "{tuple_mappings.iloc[0]["domainlabel"]}"@en] ; # domain
              wdt:{qid["exact match"]} [cidoc:{tuple_mappings.iloc[0]["cidocdomain"].replace(cidocUri, "")} ] ;}};
              wdt:{qid["range"]} {{ rdfs:label [ "{tuple_mappings.iloc[0]["rangelabel"]}"@en ] ; # range
              wdt:{qid["exact match"]} [ cidoc:{tuple_mappings.iloc[0]["cidocrange"].replace(cidocUri, "")} ]  ;}};
              wdt:{qid["property"]} {{ rdfs:label [ "{tuple_mappings.iloc[0]["propertylabel"]}"@en ] ; # property
              wdt:{qid["exact match"]} [cidoc:{tuple_mappings.iloc[0]["cidocproperty"].replace(cidocUri, "")}]  ;}};
        }}
        """
    print(prop_id)

0
P1
0
P2
0
P3
0
P4
0
P5
0
P6
0
P7
1
P8
1
P10
1
P11
0
P12
1
P13
1
P9
0
P14
0
P16
1
P17
1
P18
1
P19
1
P20
1
P21
0
P22
1
P23
1
P24
1
P25
1
P26
0
P27
0
P28
1
P29
1
P30
1
P31
1
P32
1
P33
1
P34
1
P35
0
P36
0
P37
0
P38
0
P39
0
P40
0
P41
0
P42
0
P43
0
P44
0
P45
0
P46
0
P47
0
P48
0
P49
0
P50


In [79]:
print(shexOutput)


        PREFIX wdt: <https://lux-saf-beta.wiki.opencura.com/prop/direct/>
        PREFIX wd: <https://lux-saf-beta.wiki.opencura.com/entity/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX saflux: <https://lux-saf-beta.wiki.opencura.com/entity/>
        PREFIX p: <https://lux-saf-beta.wiki.opencura.com/prop/>
        PREFIX cidoc: <http://erlangen-crm.org/current/>

            wdt:P1 {
            rdfs:label ["ass eng"@lb "instance of"@en "ist ein(e)"@de "instance de"@fr]+;
            }
            wdt:P2 {
            rdfs:label ["Ënnerklass vu(n)"@lb "subclass of"@en "Unterklasse von"@de "sous-classe de"@fr]+;
            }
            wdt:P3 {
            rdfs:label ["genauen Match"@lb "exact match"@en "exakte Übereinstimmung"@de "correspondance exacte"@fr]+;
            }
            wdt:P4 {
            rdfs:label ["domain"@lb "domain"@en "domain"@de "domaine"@fr]+;
            }
            wdt:P5 {
            rdfs:label ["reechwäit"@lb "range"@en

In [17]:
mappings

,wikibaseprop,domain,range,property,cidocdomain,cidocrange,cidocproperty,domainlabel,rangelabel,propertylabel
0,P10,Q41,Q53,Q896,http://erlangen-crm.org/current/E41_Appellation,http://erlangen-crm.org/current/E90_Symbolic_O...,http://erlangen-crm.org/current/R8_consists_of,E41 Appellation,E90 Symbolic Object,R8 consists of
1,P11,Q41,Q53,Q896,http://erlangen-crm.org/current/E41_Appellation,http://erlangen-crm.org/current/E90_Symbolic_O...,http://erlangen-crm.org/current/R8_consists_of,E41 Appellation,E90 Symbolic Object,R8 consists of
2,P13,Q37,Q70,Q158,http://erlangen-crm.org/current/E31_Document,http://erlangen-crm.org/current/E55_Type,http://erlangen-crm.org/current/P2_has_type,E31 Document,E55 Type,P2 has type
3,P17,Q41,Q53,Q896,http://erlangen-crm.org/current/E41_Appellation,http://erlangen-crm.org/current/E90_Symbolic_O...,http://erlangen-crm.org/current/R8_consists_of,E41 Appellation,E90 Symbolic Object,R8 consists of
4,P18,Q42,Q21,Q106,http://erlangen-crm.org/current/E67_Birth,http://erlangen-crm.org/current/E52_Time-Span,http://erlangen-crm.org/current/P4_has_time-span,E67 Birth,E52 Time-Span,P4 has time-span
5,P19,Q42,Q69,Q205,http://erlangen-crm.org/current/E67_Birth,http://erlangen-crm.org/current/E53_Place,http://erlangen-crm.org/current/P7_took_place_at,E67 Birth,E53 Place,P7 took place at
6,P20,Q66,Q21,Q106,http://erlangen-crm.org/current/E69_Death,http://erlangen-crm.org/current/E52_Time-Span,http://erlangen-crm.org/current/P4_has_time-span,E69 Death,E52 Time-Span,P4 has time-span
7,P21,Q66,Q69,Q205,http://erlangen-crm.org/current/E69_Death,http://erlangen-crm.org/current/E53_Place,http://erlangen-crm.org/current/P7_took_place_at,E69 Death,E53 Place,P7 took place at
8,P23,Q76,Q70,Q158,http://erlangen-crm.org/current/E7_Activity,http://erlangen-crm.org/current/E55_Type,http://erlangen-crm.org/current/P2_has_type,E7 Activity,E55 Type,P2 has type
9,P24,Q76,Q21,Q106,http://erlangen-crm.org/current/E7_Activity,http://erlangen-crm.org/current/E52_Time-Span,http://erlangen-crm.org/current/P4_has_time-span,E7 Activity,E52 Time-Span,P4 has time-span


In [70]:
shexOutput = f"""
PREFIX saflux: <{wikibase}entity/>
PREFIX p: <{wikibase}prop/>
PREFIX ps: <{wikibase}prop/statement/>  
PREFIX pq: <{wikibase}prop/qualifier/>
PREFIX pr: <{wikibase}prop/reference/>
PREFIX wdt: <{wikibase}prop/direct/>
PREFIX wd: <{wikibase}entity/>
PREFIX cidoc: <{cidocUri}>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

shexOutput += f"""
saflux:person {{
  p:{qid["name"]} @saflux:{qid["name"]}_name ; 
  p:{qid["alternative name"]} @saflux:{qid["alternative name"]}_alternative_name* ;
  p:{qid["place of birth"]} @saflux:{qid["place of birth"]}_place_of_birth ;
  p:{qid["date of birth"]} @saflux:{qid["date of birth"]}_date_of_birth ;
  p:{qid["place of death"]} @saflux:{qid["place of death"]}_place_of_death ;
  p:{qid["date of death"]} @saflux:{qid["date of death"]}_date_of_death ;
  p:{qid["gender"]} @saflux:{qid["gender"]}_gender ;
  p:{qid["profession"]} @saflux:{qid["profession"]}_profession ;
  p:{qid["activity"]} @saflux:{qid["activity"]}_activity + ;
  p:{qid["AFL identifier"]} @saflux:{qid["AFL identifier"]}_AFL_identifier ;
  p:{qid["ISNI"]} @saflux:{qid["ISNI"]}_ISNI ;
  p:{qid["VIAF"]} @saflux:{qid["VIAF"]}_VIAF ;
  p:{qid["ARK"]} @saflux:{qid["ARK"]}_ARK ;
  
}}

saflux:{qid["name"]}_name {{
  ps:{qid["name"]} xsd:string ;
  pq:{qid["numeration"]} xsd:string ;
  pq:{qid["title"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference +
}}

saflux:{qid["alternative name"]}_alternative_name {{
  ps:{qid["alternative name"]} xsd:string ;
  pq:{qid["numeration"]} xsd:string ;
  pq:{qid["title"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference +
}}

saflux:{qid["place of birth"]}_place_of_birth {{
  ps:{qid["place of birth"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["date of birth"]}_date_of_birth {{
  (ps:{qid["date of birth"]} xsd:edtf,
   ps:{qid["date of birth"]} xsd:dateTime+) ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["place of death"]}_place_of_death {{
  ps:{qid["place of death"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["date of death"]}_date_of_death {{
  (ps:{qid["date of birth"]} xsd:edtf,
   ps:{qid["date of birth"]} xsd:dateTime+ )
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["gender"]}_gender {{
  ps:{qid["gender"]} [
                        wd:{qid["male"]} 
                        wd:{qid["female"]}
                        wd:{qid["not known"]}
                        wd:{qid["not applicable"]}
                     ]
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["profession"]}_profession {{
  ps:{qid["profession"]} xsd:string ; 
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["activity"]}_activity {{
  ps:{qid["activity"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["AFL identifier"]}_AFL_identifier {{
  ps:{qid["AFL identifier"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["ISNI"]}_ISNI {{
  ps:{qid["ISNI"]} /[0-9]4 [0-9]4 [0-9]4 [0-9]4/ ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["VIAF"]}_VIAF {{
  ps:{qid["VIAF"]} xsd:string ;
  prov:wasDerivedFrom @saflux:reference + ;
}}
saflux:{qid["ARK"]}_ARK {{
  ps:{qid["ARK"]} IRI ;
  prov:wasDerivedFrom @saflux:reference + ;
}}

saflux:reference {{
 (pr:{qid["source of information - url"]} IRI) OR  
 (pr:{qid["source of information - text"]} xsd:string) OR 
 (pr:{qid["source of information - url"]} IRI ; pr:{qid["source of information - text"]} xsd:string)
}}

"""

In [71]:
print(shexOutput)


PREFIX saflux: <https://lux-saf-beta.wiki.opencura.com/entity/>
PREFIX p: <https://lux-saf-beta.wiki.opencura.com/prop/>
PREFIX ps: <https://lux-saf-beta.wiki.opencura.com/prop/statement/>  
PREFIX pq: <https://lux-saf-beta.wiki.opencura.com/prop/qualifier/>
PREFIX pr: <https://lux-saf-beta.wiki.opencura.com/prop/reference/>
PREFIX wdt: <https://lux-saf-beta.wiki.opencura.com/prop/direct/>
PREFIX wd: <https://lux-saf-beta.wiki.opencura.com/entity/>
PREFIX cidoc: <http://erlangen-crm.org/current/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

saflux:person {
  p:P8 @saflux:P8_name ; 
  p:P17 @saflux:P17_alternative_name* ;
  p:P19 @saflux:P19_place_of_birth ;
  p:P18 @saflux:P18_date_of_birth ;
  p:P21 @saflux:P21_place_of_death ;
  p:P20 @saflux:P20_date_of_death ;
  p:P22 @saflux:P22_gender ;
  p:P23 @saflux:P23_profession ;
  p:P26 @saflux:P26_activity + ;
  p:P29 @saflux:P29_AFL_identi